In [1]:
import time
import numpy as np
import torch

from get_args import get_args
from agent import Agent
from utils import load_filepath, load_checkpoint, plot_scores, print_debug_info, print_status, save_checkpoint

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from unityagents import UnityEnvironment

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
args = get_args()
args.framework = "DDQN"

In [6]:
for arg in vars(args):
    print("{}: {}".format(arg, getattr(args, arg)))

framework: DDQN
prioritized_replay: False
alpha: 0.4
batchsize: 64
buffersize: 100000
c: 8
continue: False
debug: False
dropout: 0.05
epsilon: 1.0
epsilon_decay: 0.999
epsilon_min: 0.075
gamma: 0.99
latest: False
learn_rate: 0.00025
momentum: 0.95
nographics: False
num_episodes: 1500
optimizer: Adam
pixels: False
print_count: 15
tau: 0.001
train: False
update_every: 4
verbose: False


In [7]:
start_time = time.time()
sep = "#"*50
unity_env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe", no_graphics=True)
# get the default brain (In this environment there is only one agent/brain)
brain_name = unity_env.brain_names[0]
brain = unity_env.brains[brain_name]
env = unity_env.reset(train_mode=True)[brain_name]
nA = brain.vector_action_space_size
nS = len(env.vector_observations[0])

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

In [ ]:
agent = Agent(nS, nA, device, args)

In [ ]:
env = unity_env.reset(train_mode=args.train)[brain_name]
state = env.vector_observations[0]


In [ ]:
for i in range(66):
    action = agent.act(state)
    env = unity_env.step(action)[brain_name]

    #collect info about new state
    reward = env.rewards[0]
    next_state = env.vector_observations[0]
    done = env.local_done[0]

    agent.teststep(state, action, reward, next_state, done)
    agent.memory.add(state, action, reward, next_state, done)

    state = next_state
    

In [ ]:
batch = agent.memory.sample()
#batch = state, action, reward, next_state, done

In [ ]:
for state, action, reward, next_state, done in zip(*batch):
    print("STATE:\n{}".format(state))
    print("ACTION:\n{}".format(action))
    print("REWARD:\n{}".format(reward))
    print("NEXT STATE:\n{}".format(next_state))
    print("DONE:\n{}".format(done))    

In [ ]:
print("STATE:\n{}".format(state))
print("ACTION:\n{}".format(action))
print("REWARD:\n{}".format(reward))
print("NEXT STATE:\n{}".format(next_state))
print("DONE:\n{}".format(done))

In [ ]:
#Binary tree
class Node:
    def __init__(self,data)